In [1]:
import numpy as np
from matplotlib import cm, pyplot as plt
from utils import from_run_kength_enc
import itertools
from pathlib import Path
import csv
from skimage import io
from random import shuffle


In [5]:
data_dir = "data/stage2_test_final"
submission_dir = "h5_files/submissions"
submission_filename = None
save_submitted_masks = True
colors = cm.get_cmap().colors
shuffle(colors)

In [6]:
def get_color_gen():
    return (colors[n%len(colors)] for n in itertools.count())

In [7]:
submission_dir = Path(submission_dir)

submission_path = None 

if submission_filename is None:
    submission_paths = sorted([submission_path for submission_path in submission_dir.iterdir() if submission_path.suffix == ".csv"])
    submission_path = submission_paths[-1]
    submission_filename = submission_path.name
    
    print("reading submission : {}".format(submission_path))

else:
    submission_path = submission_dir/submission_filename

submission_name = submission_filename[:-len(submission_path.suffix)]

data_dir = Path(data_dir)

reading submission : h5_files/submissions/2019.04.28-14.17.csv


In [8]:
with open(str(submission_path)) as submission_file:
    csv_reader = csv.reader(submission_file)
    current_image_id = None
    current_mask = None
    header = csv_reader.__next__()
    number_of_no_nuclei = 0
    number_of_images = 0
    for i, row in enumerate(csv_reader):
        if not i % 1000:
            print("{} nuclei treated".format(i))
        image_id = row[0]
        run_length_str = row[1]
        
        run_length = [int(elem) for elem in run_length_str.strip().split(" ") if elem.strip()]
        if not run_length:
            if current_image_id is not None and current_image_id != image_id:
                print("no segmentation for image {}".format(image_id))
                number_of_no_nuclei += 1
            else:
                print("nuclei with no pixel for image {}".format(image_id))
        # print("coucou : {}".format(image_id))
        if current_image_id is not None and current_image_id != image_id:
            if save_submitted_masks:
                submission_mask_path = data_dir / current_image_id / "submissions" / (submission_name + ".png")
                # print(submission_mask_path)
                submission_mask_path.parent.mkdir(parents=True, exist_ok=True)
                # print(current_mask.dtype)
                current_mask = (current_mask * 256).astype(np.uint8)
                io.imsave(submission_mask_path, current_mask, check_contrast=False)
            current_mask = None
        
        if current_mask is None:
            current_image_id = image_id
            image_path = data_dir / image_id / "images" / (image_id + ".png")
            image = io.imread(image_path)
            current_mask = np.zeros((*image.shape[:2], 3), dtype=np.float32)
            color_gen = get_color_gen()
            number_of_images += 1
        
        mask = from_run_kength_enc(run_length, image.shape[:2])
        
        current_mask[mask.astype(bool)] = np.array(color_gen.__next__())
        if np.sum(current_mask > 1):
            print(current_mask[mask.astype(bool)])
        

0 nuclei treated
1000 nuclei treated
2000 nuclei treated
3000 nuclei treated
4000 nuclei treated
5000 nuclei treated
6000 nuclei treated
7000 nuclei treated
8000 nuclei treated
9000 nuclei treated
10000 nuclei treated
11000 nuclei treated
12000 nuclei treated
13000 nuclei treated
14000 nuclei treated
15000 nuclei treated
16000 nuclei treated
17000 nuclei treated
18000 nuclei treated
19000 nuclei treated
20000 nuclei treated
21000 nuclei treated
22000 nuclei treated
23000 nuclei treated
24000 nuclei treated
25000 nuclei treated
26000 nuclei treated
27000 nuclei treated
28000 nuclei treated
29000 nuclei treated
30000 nuclei treated
31000 nuclei treated
32000 nuclei treated
33000 nuclei treated
34000 nuclei treated
35000 nuclei treated
36000 nuclei treated
37000 nuclei treated
38000 nuclei treated
39000 nuclei treated
40000 nuclei treated
41000 nuclei treated
42000 nuclei treated
43000 nuclei treated
44000 nuclei treated


In [6]:
print("number of no nuclei  {} out of {}".format(number_of_no_nuclei, number_of_images) )

number of no nuclei  264 out of 3019


In [7]:
from time import time
print("{:.3}".format(3.233445))

3.23
